- Diabetic           yes
- Race               other
- Sex                indefinido
- GenHealth          Primero arreglar los ya definidos(Very good/Good/Excellent/Fair/Poor), los que no hacen parte del grupo 
cambiar a undefined, sacar moda y cambiar undefined a moda.
- Smoking            yes
- Stroke             yes
- BMI                promedio dependiendo del genero 



In [1]:
import pickle as pkl
import numpy as np
import pandas as pd

In [2]:
dataOriginal = pd.read_csv('data\heart_disease_data.csv')

In [3]:
data = dataOriginal[pd.notnull(dataOriginal['HeartDisease'])]

In [4]:
dataOriginal.shape

(67500, 19)

In [5]:
data.shape

(61874, 19)

In [6]:
data.describe()

,Index,SleepTime
count,61874.000000,61874.000000
mean,33783.843779,10.233895
std,19500.423273,26.096329
min,0.000000,-20.000000
25%,16885.250000,6.000000
50%,33829.500000,7.000000
75%,50690.750000,8.000000
max,67499.000000,406.000000


In [7]:
data.head()

,Index,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
0,0,26.5,No,No,No,0.0,0.0,No,Male,64-60,White,No,Yes,Very good,8.0,No,No,No,No
1,1,28.75,ýNoÚ,No,No,0.0,2.0,No,Male,59-55,White,No,Yes,Very good,7.0,No,No,No,No
2,2,19.9,Yes,No,No,0.0,1.0,No,Male,39-35,White,No,Yes,Good,7.0,No,No,No,No
3,3,28.06,No,No,No,1.0,1.0,No,Female,69-65,White,Yes,Yes,Good,8.0,No,No,No,£NoÇ
4,4,29.29,Yes,No,No,0.0,1.0,No,Female,74-70,White,No,Yes,Very good,7.0,Yes,No,No,No


In [8]:
data.dtypes

Index                 int64
BMI                  object
Smoking              object
AlcoholDrinking      object
Stroke               object
PhysicalHealth       object
MentalHealth         object
DiffWalking          object
Sex                  object
AgeCategory          object
Race                 object
Diabetic             object
PhysicalActivity     object
GenHealth            object
SleepTime           float64
Asthma               object
KidneyDisease        object
SkinCancer           object
HeartDisease         object
dtype: object

In [9]:
def clean_yes_no(column):
    data[column].fillna("Yes", inplace = True)
    data.loc[data[column].str.contains('Yes'), column] = "Yes"
    data.loc[data[column].str.contains('No'), column] = "No"
    data.loc[~data[column].str.contains('No'), column] = "Yes"
    

In [10]:
dañadas = ['Smoking', 'Stroke', 'HeartDisease', 'Diabetic']
for col in dañadas:
    clean_yes_no(col)

C:\Users\elpin\AppData\Local\Temp\ipykernel_23144\3272468483.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column].fillna("Yes", inplace = True)


In [11]:
for col in data.columns:
    if data[col].dtype == 'object':
        print(col, data[col].unique())

BMI ['26.5' '28.75' '19.9' ... '22.88' '23.69wj' '243.60000000000002']
Smoking ['No' 'Yes']
AlcoholDrinking ['No' 'Yes']
Stroke ['No' 'Yes']
PhysicalHealth ['0.0' '1.0' '14.0' '3.0' '-0.0' '2.0' '5.0' '30.0' '10.0' '693.0' '25.0'
 '12.0' '20.0' '15.0' '4.0' '0.0umi' '0.0bhru' '21.0' '29.0' '0.0cnu'
 '448.0' '7.0' '0.0pt' '28.0' '-5.0' '0.0a' '18.0' '13.0' '64.0' '27.0'
 '-4.0' '6.0' '0.0q' '28.0vz' '156.0' '9.0' '93.0' '16.0' '0.0aob' '3.0so'
 '0.0tq' '0.0g' '2.0behj' '0.0pyam' '175.0' '0.0vyj' '105.0' '10.0ameh'
 '8.0' '0.0qjrs' '150.0' '15.0l' '-1.0' '930.0' '4.0zffk' '-2.0' '0.0jror'
 '26.0' '22.0' '0.0o' '62.0' '0.0vy' '1020.0' '28.0m' '2.0lga' '114.0'
 '-14.0' '0.0asel' '120.0' '0.0z' '465.0' '3.0i' '0.0ytgc' '19.0'
 '0.0qsih' '0.0se' '0.0kirk' '24.0' '546.0' '17.0' '8.0j' '0.0cmj'
 '1170.0' '-30.0' '0.0lytd' '5.0q' '0.0frq' '0.0j' '11.0' '0.0vguk'
 '0.0phzv' '0.0pi' '0.0kldf' '23.0' '0.0ez' '0.0v' '0.0ar' '34.0' '0.0pqz'
 '-3.0' '2.0ppod' '0.0siw' '-12.0' '0.0qsud' '-21.0' '-22.0